<a href="https://colab.research.google.com/github/Shakil-1501/S4/blob/master/Copy_of_EVA4_Session_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        #self.conv1 = nn.Conv2d(1, 8, 3) #input -? OUtput? RF
        #self.conv2 = nn.Conv2d(8, 8, 3)
        #self.pool1 = nn.MaxPool2d(2, 2)
        #self.conv3 = nn.Conv2d(8,16, 3)
        #self.conv4 = nn.Conv2d(16,32, 1)
        #self.pool2 = nn.MaxPool2d(2, 2)
        #self.conv5 = nn.Conv2d(32,8, 3)
        #self.conv6 = nn.Conv2d(8,64, 1)
        #self.conv7 = nn.Conv2d(64,10, 3)

        self.conv1 = nn.Conv2d(1, 8, 3) #input -? OUtput? RF
        self.conv2 = nn.Conv2d(8, 8, 3)
        self.bat1  = nn.BatchNorm2d(8)
        
        self.pool1 = nn.MaxPool2d(2, 2)
        #self.dropout1 = nn.Dropout2d(0.05)
        self.conv3 = nn.Conv2d(8,16, 3)
        self.conv4 = nn.Conv2d(16,16, 1)
        self.bat2  = nn.BatchNorm2d(16)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.dropout2 = nn.Dropout2d(0.1)
        self.conv5 = nn.Conv2d(16,64, 3)
        self.conv6 = nn.Conv2d(64,32, 1)
        self.conv7 = nn.Conv2d(32,10, 3)

    def forward(self, x):
        x = F.relu(self.conv2(F.relu(self.conv1(x))))
        x = self.pool1(F.relu(self.bat1(x)))
        #x = self.dropout1(x)
        x = F.relu(self.conv4(F.relu(self.conv3(x))))
        x = self.pool2(F.relu(self.bat2(x)))
        x = self.dropout2(x)
        x = F.relu(self.conv6(F.relu(self.conv5(x))))
        x = self.conv7(x)
        x = x.view(-1, 10)
        return F.log_softmax(x)


In [0]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 26, 26]              80
            Conv2d-2            [-1, 8, 24, 24]             584
       BatchNorm2d-3            [-1, 8, 24, 24]              16
         MaxPool2d-4            [-1, 8, 12, 12]               0
            Conv2d-5           [-1, 16, 10, 10]           1,168
            Conv2d-6           [-1, 16, 10, 10]             272
       BatchNorm2d-7           [-1, 16, 10, 10]              32
         MaxPool2d-8             [-1, 16, 5, 5]               0
         Dropout2d-9             [-1, 16, 5, 5]               0
           Conv2d-10             [-1, 64, 3, 3]           9,280
           Conv2d-11             [-1, 32, 3, 3]           2,080
           Conv2d-12             [-1, 10, 1, 1]           2,890
Total params: 16,402
Trainable params: 16,402
Non-trainable params: 0
---------------------------------

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:39: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [0]:


torch.manual_seed(1)
batch_size = 30

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [0]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [0]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 18):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/2000 [00:00<?, ?it/s]


Test set: Average loss: 0.0704, Accuracy: 9774/10000 (97.74%)



  0%|          | 0/2000 [00:00<?, ?it/s]


Test set: Average loss: 0.0515, Accuracy: 9847/10000 (98.47%)



  0%|          | 0/2000 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0248, Accuracy: 9933/10000 (99.33%)



  0%|          | 0/2000 [00:00<?, ?it/s]


Test set: Average loss: 0.0253, Accuracy: 9920/10000 (99.20%)



loss=0.002098306082189083 batch_id=1999: 100%|██████████| 2000/2000 [00:20<00:00, 97.02it/s]



Test set: Average loss: 0.0241, Accuracy: 9936/10000 (99.36%)

